In [ ]:
import numpy as np
import os
import tensorflow as tf
import cv2
import imutils
import matplotlib.patches as patches

from matplotlib import pyplot as plt

from collections import namedtuple
from collections import defaultdict

from skimage import segmentation
from skimage import measure
from imutils import perspective
from pylab import array, plot, show, axis, arange, figure, uint8 

from utils import ops as utils_ops
from utils import label_map_util
from utils import visualization_utils as vis_util

tf.debugging.set_log_device_placement(True)

print ('Tensorflow version:', tf.__version__)

In [ ]:
MODEL_FOLDER = 'exported_model_directory'

# Change it to load different models from MODEL_FOLDER
PATH_TO_PLATE_DETECTOR = MODEL_FOLDER + '/plate_detector_ssd_inception_v2/frozen_inference_graph.pb'

PATH_TO_PLATE_LABELS = os.path.join('label_map', 'plate_label_map.pbtxt')

In [ ]:
# Load a (frozen) Tensorflow model into memory
plate_detection_graph = tf.Graph()
with plate_detection_graph.as_default():
    od_graph_def1 = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_PLATE_DETECTOR, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def1.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def1, name='')

In [ ]:
# Loading label map
plate_category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_PLATE_LABELS, use_display_name=True)

In [ ]:
# Helper code
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
# Loading the test images
PATH_TO_TEST_IMAGES_DIR = 'mongol_alpr_images/'

TEST_IMAGE_PATHS = []
for file_name in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    fformat = file_name.split('.')[1]
    if fformat == 'jpg' or fformat == 'png':
        TEST_IMAGE_PATHS.append(PATH_TO_TEST_IMAGES_DIR + file_name)

# Size, in inches, of the output images.
IMAGE_SIZE = (9, 6)

In [ ]:
def run_inference_for_images(images, graph):
    with graph.as_default():
        with tf.Session() as sess:
            output_dicts = []
            for i, image in enumerate(images):
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                    'num_detections', 'detection_boxes', 'detection_scores',
                    'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
                if 'detection_masks' in tensor_dict:
                    # The following processing is only for single image
                    detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                    detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                    # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                    real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                    detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                    detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                        detection_masks, detection_boxes, image.shape[1], image.shape[2])
                    detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                    # Follow the convention by adding back the batch dimension
                    tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
                image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

                # Run inference
                output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image})

                # all outputs are float32 numpy arrays, so convert types as appropriate
                output_dict['num_detections'] = int(output_dict['num_detections'][0])
                output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.int64)
                output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
                output_dict['detection_scores'] = output_dict['detection_scores'][0]
                if 'detection_masks' in output_dict:
                    output_dict['detection_masks'] = output_dict['detection_masks'][0]
                output_dicts.append(output_dict)
                
                print (str(int(100*(i/len(images)))) + '%', end='\r')
            return output_dicts

In [ ]:
def char_segment(im):
    # Removing shadow from an image
    # https://stackoverflow.com/questions/44752240/how-to-remove-shadow-from-scanned-images-using-opencv
    rgb_planes = cv2.split(im)
    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((3,3), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 31)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)

    result = cv2.merge(result_planes)
    result_norm = cv2.merge(result_norm_planes)
    
    # Create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    # Applying a contrast limited adaptive histogram equlization
    im = clahe.apply(im)
    
    # Turning the image into binary image
    ret, im = cv2.threshold(im,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    im = segmentation.clear_border(im)
    
    # Finding the bounding boxes of the label masks
    bboxes = []
    
    labels = measure.label(im, neighbors=8, background=0)
    charCandidates = np.zeros(im.shape, dtype="uint8")

    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue

        # otherwise, construct the label mask to display only connected components for the
        # current label, then find contours in the label mask
        labelMask = np.zeros(im.shape, dtype="uint8")
        labelMask[labels == label] = 255
        cnts = cv2.findContours(labelMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
#         cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]

        # grab the largest contour which corresponds to the component in the mask, then
        c = max(cnts, key=cv2.contourArea)
            
        # grab the bounding box for the contour
        (boxX, boxY, boxW, boxH) = cv2.boundingRect(c)

        # compute the aspect ratio, solidity, and height ratio for the component
        aspectRatio = boxW / float(boxH)
        solidity = cv2.contourArea(c) / float(boxW * boxH)
        areaRatio = float(boxW * boxH) / float(im.shape[0] * im.shape[1])
        heightRatio = boxH / float(im.shape[0])

        bboxes.append([boxX-1, boxY-1, boxW+1, boxH+1, aspectRatio, areaRatio])
    
    return bboxes

In [ ]:
LP_labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: '9',
    10: 'a',
    11: 'b',
    12: 'ch',
    13: 'd',
    14: 'e',
    15: 'g',
    16: 'h',
    17: 'i',
    18: 'k',
    19: 'l',
    20: 'm',
    21: 'n',
    22: 'p',
    23: 'q',
    24: 'r',
    25: 's',
    26: 't',
    27: 'ts',
    28: 'u',
    29: 'uu',
    30: 'v',
    31: 'ya',
    32: 'ye',
    33: 'z'
}
# returns (Character string, probability)
def decode_output(output):
    output = list(output)
    the_max = max(output)
    return (LP_labels[output.index(the_max)], the_max)

In [ ]:
char_binary_model = tf.keras.models.load_model('char_recognition_models/plate_char_binary_recognizer.h5')
char_model = tf.keras.models.load_model('char_recognition_models/plate_char_recognizer.h5')

In [ ]:
# get frames from video
def get_frames_from_vid(video_path):
    frames = []
    vidcap = cv2.VideoCapture(video_path)
    success,image = vidcap.read()
    frames.append(image)
    count = 0
    while success:
        success,image = vidcap.read()
        if success:
            frames.append(image)
        count += 1
    print (frames[0].shape)
    return frames

In [ ]:
# Loading images
images_expanded = []
images = []

for image_path in TEST_IMAGE_PATHS:
    image_np = cv2.imread(image_path, 0)
    
#     Enhancing image
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    image_np = clahe.apply(image_np)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_GRAY2BGR)
    
#     Expand dimensions since the model expects images to have a shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    images.append(image_np)
    images_expanded.append(image_np_expanded)

print (len(images), 'image(s) found')

In [ ]:
# Detection of license plates.
# This might take some resources to initialize the tf graph
# After it's done it will start detecting plates from given images
# will print the progress in percentage

plate_output_dicts = run_inference_for_images(images_expanded, plate_detection_graph)
print ('done')

In [ ]:
# This is needed to display the images.
%matplotlib inline
print (len(plate_output_dicts))

font = cv2.FONT_HERSHEY_SIMPLEX 
fontScale = 1
color = (0, 0, 255) 
thickness = 2

for i, plate_output_dict in enumerate(plate_output_dicts):
    img_height, img_width, img_channel = images[i].shape
    absolute_coord = []
    
    # ADJUST THE THRESHOLD FOR PLATE DETECTION
    THRESHOLD = 0.5
    
    N = len(plate_output_dict['detection_boxes'])
    for j in range(N):
        if plate_output_dict['detection_scores'][j] > THRESHOLD:
            box = plate_output_dict['detection_boxes'][j]
            ymin, xmin, ymax, xmax = box
            x_up = int(xmin*img_width)
            y_up = int(ymin*img_height)
            x_down = int(xmax*img_width)
            y_down = int(ymax*img_height)
            absolute_coord.append((x_up,y_up,x_down,y_down))

#     Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        images[i],
        plate_output_dict['detection_boxes'],
        plate_output_dict['detection_classes'],
        plate_output_dict['detection_scores'],
        plate_category_index,
        min_score_thresh = THRESHOLD,
        instance_masks=plate_output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=16)
    
    for c in absolute_coord:
        # cropped plate image
        plate = images[i][c[1]:c[3], c[0]:c[2],:]
        plate = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
        # bounding boxes for characters
        bboxes = char_segment(plate)
        bboxes = [bbox for bbox in bboxes if bbox[4] < 1 and bbox[4] > 0.20 and bbox[5] > 0.01 and bbox[5] < 0.15]
        
        bboxes_with_chars = []
        # recognizing characters inside bbox nad putting it in 
        for bbox in bboxes:
            # cropping potential character image
            char_img = images[i][c[1]+bbox[1]:c[1]+bbox[1]+bbox[3], c[0]+bbox[0]:c[0]+bbox[0]+bbox[2]]
            
            # padding the image by 10%
            top = int(0.1 * char_img.shape[0])
            bottom = top
            left = int(0.1 * char_img.shape[1])
            right = left
            char_img = cv2.copyMakeBorder(char_img, top, bottom, left, right, cv2.BORDER_REPLICATE)
            
            # resizing and reshaping to fit it into CNN
            char_img = cv2.resize(char_img, (20,36), interpolation = cv2.INTER_AREA)
            char_img = cv2.cvtColor(char_img, cv2.COLOR_BGR2GRAY)
            char_img = char_img.reshape(1, char_img.shape[0], char_img.shape[1], 1)
            
            # predicting character from a prepared character image
            the_char = char_model.predict(char_img)
            
            # decoding output vector into a string character
            the_char, score = decode_output(the_char[0])
            
            # drawing bbox and a character into an original image (with cars)
            images[i] = cv2.putText(images[i], str(the_char), (c[0]+bbox[0],c[1]+bbox[1]),
                                    font, fontScale, color, thickness, cv2.LINE_AA)
            
            images[i] = cv2.rectangle(images[i], (c[0]+bbox[0],c[1]+bbox[1]),
                                      (c[0]+bbox[0]+bbox[2],c[1]+bbox[1]+bbox[3]), (255,0,0), 2)
            
            bboxes_with_chars.append([bbox, the_char])
    cv2.imwrite('mongol_alpr_output_test/'+'{}.jpg'.format(i), images[i])
    print (str(int(100*(i/len(plate_output_dicts)))) + '%', end='\r')
print ('done')